In [ ]:
import io, requests, pandas as pd

MAP_KEY = "set your key here"  # obtenha no site do FIRMS
WEST, SOUTH, EAST, NORTH = -47.99, -21.37, -47.64, -21.06  # Ribeirão Preto (WGS84)
DAY_RANGE = 3  # 1..10 dias (últimos N dias)
SENSORS = ["VIIRS_SNPP_NRT", "VIIRS_NOAA20_NRT", "VIIRS_NOAA21_NRT"]  # pode incluir "MODIS_NRT"

BASE = "https://firms.modaps.eosdis.nasa.gov/api/area/csv"
COORDS = f"{WEST},{SOUTH},{EAST},{NORTH}"

def fetch_sensor(sensor, date=None):
    url = f"{BASE}/{MAP_KEY}/{sensor}/{COORDS}/{DAY_RANGE}"
    if date:  # opcional: data inicial específica (YYYY-MM-DD)
        url = f"{url}/{date}"
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    df = pd.read_csv(io.StringIO(r.text))
    df["sensor"] = sensor
    return df

dfs = []
for s in SENSORS:
    try:
        dfs.append(fetch_sensor(s))
    except Exception as e:
        print("Falha em", s, "->", e)

fires = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()
print(fires.shape)
fires.to_csv("fires_ribeirao_preto_last3days.csv", index=False, encoding="utf-8-sig")
fires.head()


(4, 15)


C:\Users\eduar\AppData\Local\Temp\ipykernel_6868\3964581999.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  fires = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()


,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,sensor
0,-21.08971,-47.76276,302.02,0.52,0.50,2025-10-04,355,N,VIIRS,n,2.0NRT,255.40,2.28,N,VIIRS_SNPP_NRT
1,-21.08868,-47.76781,295.71,0.52,0.50,2025-10-04,355,N,VIIRS,n,2.0NRT,260.79,1.02,N,VIIRS_SNPP_NRT
2,-21.09122,-47.76515,321.55,0.40,0.37,2025-10-04,417,N20,VIIRS,n,2.0NRT,254.96,2.91,N,VIIRS_NOAA20_NRT
3,-21.08786,-47.76452,343.05,0.40,0.37,2025-10-04,417,N20,VIIRS,n,2.0NRT,257.70,5.58,N,VIIRS_NOAA20_NRT
